In [ ]:
# 웹크롤링 절차
# 1. 웹서비스분석 : 크롬 브라우져의 개발자도구 : URL
# 2. Request(URL) > Response(Data) : Data(json)
# 3. Data(json) > list, dict > DataFrame

In [ ]:
# API : application programing interface
# 프로그램을 이용해서 데이터를 제공하는 서비스
# 제3자가 브라우져를 사용해서 데이터를 가져가는것이 아니라, 프로그램으로 데이터를 가져갈수 있도록하는 서비스
# 어플리케이션 등록 > key 값을 수령

### Naver API
- 파파고 번역 api
- 통합검색어 트렌드 api

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import requests, json

#### 0. request token 얻기
- `https://developers.naver.com`
- 1. Request Token 얻기 : 애플리케이션등록 -> app_key 획득
- 2. app_key를 이용해서 데이터 가져오기

In [7]:
client_id = 'IlukMr51VAABbSM1YL_E'
client_secret = 'ojJSn8HOgT'

In [8]:
# client_id = '7Qxcb0Evh01fdhToUREv'
# client_secret = 'y8Jv0DplVr'

In [9]:
# 어제와 다른 2가지 방법
# 1. APP 등록 : key 받아서 request를 해야함
# 2. document를 확인해서 url을 받아옴

#### 1. 파파고 번역 api
- `https://developers.naver.com/docs/papago/`
- 사용법
    - `https://developers.naver.com/docs/papago/papago-nmt-api-reference.md`

In [10]:
# 1. URL
url = 'https://openapi.naver.com/v1/papago/n2mt'

In [13]:
# headers
headers = {
    'Content-Type': 'application/json',
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret,
}

# data
ko_text = '웹크롤링은 재미있고 돈이됩니다.'
params = {'source': 'ko', 'target': 'en', 'text': ko_text}
params

{'source': 'ko', 'target': 'en', 'text': '웹크롤링은 재미있고 돈이됩니다.'}

In [17]:
# 한글 인코딩
json.dumps(params)

'{"source": "ko", "target": "en", "text": "\\uc6f9\\ud06c\\ub864\\ub9c1\\uc740 \\uc7ac\\ubbf8\\uc788\\uace0 \\ub3c8\\uc774\\ub429\\ub2c8\\ub2e4."}'

In [18]:
# 2. Request(url,headers,data) > Response
# 인터넷 환경에서 데이터를 전달할때는 영문,숫자,특수문자만 사용가능
response = requests.post(url, json.dumps(params), headers=headers)
response

<Response [200]>

In [19]:
response.text

'{"message":{"result":{"srcLangType":"ko","tarLangType":"en","translatedText":"Web crawling is fun and money.","engineType":"N2MT"},"@type":"response","@service":"naverservice.nmt.proxy","@version":"1.0.0"}}'

In [24]:
# 3. json(str) > list, dict > text
en_text = response.json()['message']['result']['translatedText']
en_text

'Web crawling is fun and money.'

In [25]:
# 4. make function
def translate(ko_text):
    
    # 0. application key
    client_id = 'IlukMr51VAABbSM1YL_E'
    client_secret = 'ojJSn8HOgT'
    
    # 1. url, headers, data
    url = 'https://openapi.naver.com/v1/papago/n2mt'
    headers = {
        'Content-Type': 'application/json',
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret,
    }
    params = {'source': 'ko', 'target': 'en', 'text': ko_text}
    
    # 2. request(url, headers, data) > response(json)
    response = requests.post(url, json.dumps(params), headers=headers)
    
    # 3. json > dict > text
    en_text = response.json()['message']['result']['translatedText']
    
    return en_text

In [26]:
translate('파이썬은 재미있습니다.')

'Python is fun.'

In [29]:
%ls covid.xlsx

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: F618-ECDB

 C:\Users\User\Desktop\web_crawling 디렉터리

2023-09-05  오전 11:18             8,911 covid.xlsx
               1개 파일               8,911 바이트
               0개 디렉터리  351,861,768,192 바이트 남음


In [33]:
# 한글엑셀파일 > 영문엑셀파일
df = pd.read_excel('covid.xlsx')
df.tail(1)

,category,title
5,104,올림픽 다가오는데 도쿄 코로나 확산 빨라져...일주일새 25％↑


In [35]:
%%time
df['en_title'] = df['title'].apply(translate)

CPU times: total: 78.1 ms
Wall time: 444 ms


In [36]:
df

,category,title,en_title
0,101,"SK바이오사이언스, 코로나19 백신 임상 3상 시험계획 제출",SK Bioscience Submits Phase 3 Trial Plan for C...
1,102,고양시 노래연습장 코로나19 누적확진 41명,41 cumulative confirmed cases of COVID-19 at t...
2,103,"코로나19 신규 감염, 28일 오후 9시까지 542명","New COVID-19 infections, 542 people by 9 p.m. ..."
3,103,프로야구 수도권 구단서 코로나19 확진자 발생,A confirmed case of COVID-19 occurred at a clu...
4,104,"""코로나 확진자 '0명'인 날은 절대 오지 않는다"" 美전문가","""The day when there are '0' confirmed cases of..."
5,104,올림픽 다가오는데 도쿄 코로나 확산 빨라져...일주일새 25％↑,"As the Olympics are approaching, the spread of..."


In [37]:
df.to_excel('en_covid.xlsx', index=False, encoding='utf-8-sig')

In [38]:
%ls en_covid.xlsx

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: F618-ECDB

 C:\Users\User\Desktop\web_crawling 디렉터리

2023-09-05  오전 11:23             5,706 en_covid.xlsx
               1개 파일               5,706 바이트
               0개 디렉터리  351,860,199,424 바이트 남음


#### 2. 통합검색어 트렌드 api
- 서비스 : `https://datalab.naver.com/keyword/trendSearch.naver`
- 내 애플리케이션 > dss 애플리케이션 > API 설정 > 데이터랩(검색어 트렌드) 추가
- 사용법 : `https://developers.naver.com/docs/serviceapi/datalab/search/search.md#통합-검색어-트렌드`

### 공공 데이터 포털
- data.go.kr
- 국토교통부_(TAGO)_카셰어링정보
- `https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15098557`

In [39]:
# 0. application key
key = '9DVRoYNb5q7dPwxtUnB1srriOEJZbGb/cOlrFAPBNTxsujahRRWPCwdcZ0KYj87rkw9Uw3GBhecH3PL3aYLiRw=='

In [41]:
# 1. url
zonename = '서울역'
url = 'http://apis.data.go.kr/1613000/CarSharingInfoService/getCarZoneListByName'
url += f'?serviceKey={key}&zoneName={zonename}&_type=json'
print(url)

http://apis.data.go.kr/1613000/CarSharingInfoService/getCarZoneListByName?serviceKey=9DVRoYNb5q7dPwxtUnB1srriOEJZbGb/cOlrFAPBNTxsujahRRWPCwdcZ0KYj87rkw9Uw3GBhecH3PL3aYLiRw==&zoneName=서울역&_type=json


In [43]:
# 2. request(url) > response(json)
response = requests.get(url)
response

<Response [200]>

In [50]:
# 3. json > list, dict > DataFrame
data = response.json()['response']['body']['items']['item']
df = pd.DataFrame(data)
df.tail(2)

,address,latitude,longitude,type,zoneId,zoneName
5,서울 용산구 동자동 56 트윈시티 남산,37.551189,126.972939,2,7426,서울역 12번출구(갈월동)
6,서울 용산구 서계동 47-2 대한통운서울지사,37.552486,126.968964,2,9442,서울역 15번출구(국립극단옆EV)
